# Sequenc to Sequence Learning with Nueral Networks
> Sequenc to Sequence Learning with Nueral Networks review

- toc: true 
- badges: true
- comments: true
- author: Bujie Xu
- categories: [NLP]

这篇文章发表于2014年底。我们所熟知的seq2seq模型由该文章提出。本文提出了一种解决由一个序列预测另一个序列问题的端到端的模型（seq2seq模型）。通过在WMT'14数据集上的测试，证明了端到端的seq2seq模型可以达到SOAT的精度。

接下来我们主要介绍以下几点：

- 背景和模型概要
- 实验设计和工程实现
- 实验结果
- 总结

## 1. 背景和模型概要

传统的DNN虽然在很多领域取得了很好的成绩，但是DNN之只能适用于输入和输出的维度都确定的情况下。对于输入和输出长度不固定的问题，传统的DNN束手无策。
在本篇文章，作者为了解决此问题，提出了一种seq2seq的模型来处理。

![seq2seq](images/seq2seq.PNG)

如上图所示，作者利用一个LSTM来编码输入序列，然后用另一个LSTM来对输出进行解码。这里用"EOS"来表示句尾，从而使得输出的序列长度由模型来决定。

## 2. 实验设计和工程实现

### 2.1 数据和评价指标


实验所采用的数据集来自WMT'14里英文到法文的翻译任务的数据。总共句子数为12M，其中法文单词有348M，英文单词有304M。另外作者事先给定了两种语言的词库，英文词库包含160,000常用词，法文词库包含80,000常用词，其他单词用"UNK"表示。

BLEU来评价机器翻译的好坏。BLEU公式如下

$$
BLEU = BP \times e^{\sum_{i=1}^nw_i\log(p_i)}
$$

$$
BP = 
\begin{cases}
1 &\text{if c>r} \\
e^{1-r/c} &c \leq r
\end{cases}
$$
这里r为一个参考长度，当翻译结果长度为r时，不需要对结果进行惩罚

### 2.2 训练详情

模型由4层LSTM作为编码器，另外一个4层LSTM作为解码器。输入的词向量维度为1000。其他实验中的细节如下

- 用一个范围为-0.08到0.08的均一分布来初始化LSTM的参数
- 用SGD进行训练，初始5个epoch的学习率设为0.7，之后每半个epoch学习率减半，总共训练了7.5个epoch
- batch size为128， 每个batch里面的句子长度尽量相近
- 为了防止梯度爆炸，采取了梯度裁剪

此外，作者还提到了一个特别重要的技巧，就是将输入进行倒排。通过实验，发现这样可以大大提升结果。从直观分析来说，这个技巧能起到作用的原因是将输出的词和输出的词在神经网络上的距离拉近了。例如在figure1里面，原来A到X的距离为4个单元，通过倒排A到X的距离变为2个单元。

在工程实现上，作者采用了8个GPU，每一层的LSTM用一个GPU计算，在前一个GPU计算完结果后会传递到下一个GPU来计算。另外4个GPU则用于softmax的并行计算。

## 3. 实验结果

实验结果如下面的table2所示，可以看出端到端的Seq2Seq模型在翻译任务上取到了比较不错的结果。提高Beam search的size和增加集成模型的各种都有助于提高模型精度。

![table1](images/seq2seq2.PNG)

另外，作者还分析了编码器对于语序和语态的表现能力。从图中可以看出模型对于语序非常敏感，但是不能很好的区分主动语态和被动语态。

![figure2](images/seq2seq3.PNG)

最后作者比较了LSTM的seq2seq模型和baseline(统计模型)在各种句子长度下的表现，发现LSTM在各种句子长度下都能有比较好的效果。
另外figure3的右图指出，在句子里常用词比较多的情况下，LSTM的表现会更好。

![figure3](images/seq2seq4.PNG)

## 4. 总结

本文提出了一种处理不固定长序列问题的端到端的方法。大大简化了以往统计建模的时间，并且精度上也达到了一个不错的级别。个人感觉，在建模上，大家可以都能想到seq2seq这种结构。但是一些trick(比如倒排输入的句子)不太容易想到，而且在工程实现上，利用多个GPU并行计算LSTM在当时也不容易做到

{{ '[Sequence to Sequence Learning with Neural Networks](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)' | fndetail: 1 }}
{{ '[自然语言处理——BLEU详解以及简单的代码实现](https://blog.csdn.net/allocator/article/details/79657792)' | fndetail: 2 }}